[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/r2.0.0-alpha/tutorials/zh_cn/advanced/modules/mindspore_initializer.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/r2.0.0-alpha/tutorials/zh_cn/advanced/modules/mindspore_initializer.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/r2.0.0-alpha/tutorials/source_zh_cn/advanced/modules/initializer.ipynb)

# 参数初始化

## 使用内置参数初始化

MindSpore提供了多种网络参数初始化的方式，并在部分算子中封装了参数初始化的功能。本节以Conv2d为例，分别介绍如何使用Initializer子类，字符串进行参数初始化。

### Initializer初始化

`Initializer`是MindSpore内置的参数初始化基类，所有内置参数初始化方法均继承该类。`mindspore.nn`中提供的神经网络层封装均提供`weight_init`、`bias_init`等入参，可以直接使用实例化的Initializer进行参数初始化。样例如下：

In [1]:
import numpy as np
import mindspore.nn as nn
import mindspore as ms
from mindspore.common.initializer import Normal, initializer

input_data = ms.Tensor(np.ones([1, 3, 16, 50], dtype=np.float32))
# 卷积层，输入通道为3，输出通道为64，卷积核大小为3*3，权重参数使用正态分布生成的随机数
net = nn.Conv2d(3, 64, 3, weight_init=Normal(0.2))
# 网络输出
output = net(input_data)

### 字符串初始化

除使用实例化的Initializer外，MindSpore也提供了参数初始化简易方法，即使用参数初始化方法名称的字符串。此方法使用Initializer的默认参数进行初始化。样例如下：

In [2]:
import numpy as np
import mindspore.nn as nn
import mindspore as ms

net = nn.Conv2d(3, 64, 3, weight_init='normal')
output = net(input_data)

## 自定义参数初始化

通常情况下，MindSpore提供的默认参数初始化可以满足常用神经网络层的初始化需求，在遇到需要自定义的参数初始化方法时，可以继承Initializer自定义参数初始化方法。下面以`XavierNormal`为例介绍自定义参数初始化方法：

In [3]:
import math
import numpy as np
from mindspore.common.initializer import Initializer, _calculate_fan_in_and_fan_out, _assignment

class XavierNormal(Initializer):
    def __init__(self, gain=1):
        super().__init__()
        # 配置初始化所需要的参数
        self.gain = gain

    def _initialize(self, arr): # arr为需要初始化的Tensor
        fan_in, fan_out = _calculate_fan_in_and_fan_out(arr.shape) # 计算fan_in, fan_out值

        std = self.gain * math.sqrt(2.0 / float(fan_in + fan_out)) # 根据公式计算std值
        data = np.random.normal(0, std, arr.shape) # 使用numpy构造初始化好的ndarray

        _assignment(arr, data) # 将初始化好的ndarray赋值到arr

完成自定义初始化方法后，我们可以像内置初始化方法一样进行调用：

In [4]:
net = nn.Conv2d(3, 64, 3, weight_init=XavierNormal())
# 网络输出
output = net(input_data)

## Cell遍历初始化

除了使用`weight_init`, `bias_init`等`mindspore.nn`接口提供的入参外，我们也习惯于先构造完整神经网络，然后对`weight`、`bias`等参数进行统一管理。此时需要先构造网络并实例化，然后对Cell进行遍历，并对参数进行赋值。下面是一个简单的样例：

In [5]:
for name, param in net.parameters_and_names():
    if 'weight' in name:
        param.set_data(initializer(Normal(), param.shape, param.dtype))
    if 'bias' in name:
        param.set_data(initializer('zeros', param.shape, param.dtype))